In [1]:
import pandas as pd
import numpy as np
import os as os
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from nltk.util import ngrams
import nltk
from arabica import arabica_freq
from unidecode import unidecode
import re
import plotly.express as px
import texthero as hero
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import NMF
from sklearn.decomposition import TruncatedSVD
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.naive_bayes import BernoulliNB
import spacy
import re
import nlp
import warnings
import statsmodels.api as sm
from statsmodels.formula.api import ols


warnings.simplefilter(action='ignore', category=FutureWarning)

2023-09-26 11:33:35.844984: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-26 11:33:35.955044: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-26 11:33:35.955641: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-26 11:33:37.670391: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
df = pd.read_csv('sentimento - base3.csv')

In [3]:
df.head()

,id_issue,mes_chat,SENTIMENTO,TIPO_MSG,hierarquia_4,ds_cluster_sociodemographic,ds_message,flag_negativo_indecx
0,15388962,2023-04-01,neutro,duvida,Vencimento fatura,"0 - Mulher, Casada, 25 anos ou mais, cidade Pe...","['Que dia minha fatura fecha?','Ainda tenho dú...",0
1,17497395,2023-07-01,neutro,duvida,Dúvidas sobre pagamento de fatura,"11 - Homem, Solteiro, Até 24 anos, cidade Pequ...","['Quero pagar os 100 reais que eu tirei','Mais...",0
2,15213047,2023-04-01,neutro,solicitacao,Não reconhecimento de compra no crédito,"17 - Mulher, 25 a 40 anos, Sudeste e Sul","['Apareceu uma compra que eu não fiz','Dúvida ...",0
3,17117301,2023-06-01,neutro,problema,Parcelamento de fatura,"8 - Mulher, Solteira, Até 24 anos, cidade Pequ...",['No meu limite está escrito que está utilizad...,0
4,17345996,2023-07-01,negativo,problema,Dúvidas PIX,"10 - Mulher, Solteira, Até 24 anos, cidade Peq...","['Pix','E merda o will nunca teve essas coisas...",0


In [4]:
len(df)

14906

In [5]:
exemplo_base = df[df['SENTIMENTO'].notnull()]
len(exemplo_base)

645

In [6]:
stopwords_will= ['da', 'em', 'você', 'de', 'ao', 'os', 'que', 'e', 'eu', 'o', 'estou', 'ele','ela','pra', 'pro', 'entao', 'para', 'voce',
                 'pra', 'tá', 'né', 'ah', 'aí', 'dá', 'ó', 'mim', 'então', 'aqui', 'tô', 'pro',
            'seu', 'sua', 'por', 'sobre', 'mais', 'uma', 'um','como', 'meu', 'com', 'outro','outra','das','dos','foi', 'fiz','pelo','tem','mas',
            'este', 'esse','ta', 'to', 'ai', 'la', 'lo', 'lhe', 'ne','pra', 'tá', 'né', 'ah', 'aí', 'dá', 'ó', 'mim', 'então', 'aqui', 'tô', 'pro', 'isso']

In [7]:
exemplo_base['SENTIMENTO_label'] = np.where(exemplo_base['SENTIMENTO'] == "negativo", 0, 
                                            np.where(exemplo_base['SENTIMENTO'] == "neutro", 1, 2 ))

/tmp/ipykernel_1055687/2720231541.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [8]:
exemplo_base['SENTIMENTO_label'].value_counts()

0    265
1    258
2    122
Name: SENTIMENTO_label, dtype: int64

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, StratifiedKFold, KFold
from sklearn import metrics
from sklearn.metrics import classification_report
import optuna.integration.lightgbm as lgb
import optuna
from sklearn.model_selection import RepeatedKFold


vetorizador = TfidfVectorizer()
X = vetorizador.fit_transform(exemplo_base['ds_message'])
y = exemplo_base.SENTIMENTO_label


x_train, x_test, y_train, y_test = train_test_split(X
                                                    , exemplo_base.SENTIMENTO_label
                                                    , random_state=42, test_size=0.20)

In [10]:
rkf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=42)

params = {
        "objective": "multiclass",
        "metric": "multi_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "num_class": 3,                
        "seed": 42
    }

study_tuner = optuna.create_study(direction='minimize')

optuna.logging.set_verbosity(optuna.logging.WARNING)

[I 2023-09-26 11:35:42,451] A new study created in memory with name: no-name-41dedc58-0754-48c8-92aa-aced8691ccc2


In [11]:
dtrain = lgb.Dataset(x_train, label=y_train)

tuner = lgb.LightGBMTunerCV(params, 
                            dtrain, 
                            #categorical_feature=ids_of_categorical,
                            study=study_tuner,
                            verbose_eval=False,                            
                            early_stopping_rounds=250,
                            time_budget=19800, # Time budget of 5 hours, we will not really need it
                            seed = 42,
                            folds=rkf,
                            num_boost_round=10000
                           )

tuner.run()

feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]/home/flavia.costa/.local/lib/python3.8/site-packages/lightgbm/engine.py:577: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
/home/flavia.costa/.local/lib/python3.8/site-packages/lightgbm/engine.py:620: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
num_leaves, val_score: 0.661066:   0%|          | 0/20 [00:00<?, ?it/s]

: 

: 

In [ ]:
print(tuner.best_params)
# Classification error
print(tuner.best_score)
# Or expressed as accuracy
print(1.0-tuner.best_score)

In [ ]:
model = lgb.LGBMClassifier(**tuner.best_params)
model.fit(x_train, y_train, eval_set=[(x_test, y_test)], eval_metric="multi_logloss", early_stopping_rounds=20,  verbose=True)

In [ ]:
y_predicted = model.predict(x_test)
pd.crosstab(y_predicted, y_test)

In [ ]:
print(classification_report(y_test, y_predicted))